# Market price monitor 
## 0. Imports

In [1]:
%matplotlib widget
import urllib.request
import json, os, sys
import mplfinance as mpf
import pandas as pd
pd.set_option('display.max_columns', 0)
import pprint
pp = pprint.PrettyPrinter(indent=4)

#sys.path.append('..')
#from mylib.helper import test2

BASEURL = 'http://localhost:18080/kabusapi/'
BASEURL_TEST = 'http://localhost:18081/kabusapi/'

## 1. Get token

In [1]:
from getpass import getpass
password = getpass()

 ········


In [4]:
json_data = json.dumps({'APIPassword': password}).encode('utf8')
req = urllib.request.Request(
    BASEURL+'token', 
    json_data, 
    method='POST'
)
req.add_header('Content-Type', 'application/json')

res = urllib.request.urlopen(req)
print(res.status, res.reason)

content = json.loads(res.read())
token = content['Token']

200 OK


## 2. Register brand

In [7]:
obj = { 'Symbols':
        [
            {'Symbol': '4689', 'Exchange': 1},
        ] }
json_data = json.dumps(obj).encode('utf8')

url = 'http://localhost:18080/kabusapi/register'
req = urllib.request.Request(url, json_data, method='PUT')
req.add_header('Content-Type', 'application/json')
req.add_header('X-API-KEY', token)


res = urllib.request.urlopen(req)
print(res.status, res.reason)
#for header in res.getheaders():
#    print(header)

content = json.loads(res.read())
pp.pprint(content)

200 OK
{'RegistList': [{'Exchange': 1, 'Symbol': '4689'}]}


## 3. Get PUSH

about PUSH: https://kabucom.github.io/kabusapi/ptal/push.html

In [7]:
def convertToPlain(dict_raw):
    dict_plain = dict_raw.copy()
    for board_type in ['Buy', 'Sell']:
        for i in range(1, 11):
            key = board_type + str(i)
            board = dict_plain.pop(key, None)
            if i == 1:
                for s in ['Sign', 'Time']:
                    dict_plain[key+'_'+s] = board[s]
            for s in ['Price', 'Qty']:
                dict_plain[key+'_'+s] = board[s]
    return dict_plain

def to_csv(df, filename):
    if not os.path.isfile(filename):
        df.to_csv(filename)
    else:
        df.to_csv(filename, mode='a', header=False)

# freq = 1s, 1min, 5min, 1h, ...
def groupByTime(df, col, freq=None):
    df[col+'Time'] = pd.to_datetime(df[col+'Time']) 
    df_sel = df.loc[:, [col+'Time', col]]
    if freq:
        df_gb = df_sel.groupby(pd.Grouper(key=col+'Time', freq=freq))
    else:
        df_gb = df_sel.groupby(col+'Time')
    return df_gb

def CreateStockData(df, freq=None):
    df_gb = groupByTime(df, 'CurrentPrice', freq=freq)
    df_vol = groupByTime(df, 'TradingVolume', freq=freq).max() 
    df_vol = df_vol.diff()
    df_stock = pd.concat([df_gb.first(), df_gb.max(), df_gb.min(), df_gb.last(), df_vol], axis=1)
    df_stock = df_stock.set_axis(['Open', 'High', 'Low', 'Close', 'Volume'], axis='columns')
    return df_stock

def mpfplot(data, **kwargs):
    style = mpf.make_mpf_style(base_mpf_style='nightclouds', rc={"font.family":'MS Gothic'})
    return mpf.plot(data, figsize=(12, 6), style=style, returnfig=True, **kwargs)

def mpfplotLive(data, **kwargs):
    style = mpf.make_mpf_style(base_mpf_style='nightclouds', rc={"font.family":'MS Gothic'})
    return mpf.plot(data, style=style, **kwargs)

df_raw = pd.DataFrame()

def livePlotFirst(df):
    df_raw = pd.concat([df_raw, df])
    df_stock = CreateStockData(df_raw, '1s')
    return mpfplot(df_stock, type='candle')

def livePlot(df, ax):
    current_price_time = pd.to_datetime(df['CurrentPriceTime'][0])
    previous_price_time = pd.to_datetime(df_raw['CurrentPriceTime'][0])
    df_raw = pd.concat([df_raw, df])
    if previous_price_time < current_price_time:
        df_stock = CreateStockData(df_raw, '1s')
        mpfplotLive(df_stock, type='candle', ax=ax[0])

In [ ]:
import json, sys, time
import websocket
from datetime import datetime

FILENAME_RAW = time.strftime("%Y%m%d") + '_push_raw.csv'
FILENAME_STOCK = time.strftime("%Y%m%d") + '_push_stock.csv'

is_first_data = True

def on_message(ws, message):
    print(datetime.now(), ', --- RECV MSG. ---', end='\r')
    #print(message) 
    # String --> dict
    dict_raw = json.loads(message)

    # Convert to plain
    dict_plain = convertToPlain(dict_raw)

    # dict --> dataframe
    df_tick = pd.DataFrame([dict_plain])
    # save raw data
    to_csv(df_tick, FILENAME_RAW)
    
    if is_first_data:
        fig, ax = livePlotFirst(df_tick)
        is_first_data = False
    else:
        livePlot(df_tick, ax)
        fig.canvas.draw()

def on_error(ws, error):
    print('--- ERROR --- ')
    print(error)

def on_close(ws):
    print('--- DISCONNECTED --- ')

def on_open(ws):
    print('--- CONNECTED --- ')

url = 'ws://localhost:18080/kabusapi/websocket'
# websocket.enableTrace(True)
ws = websocket.WebSocketApp(
    url,
    on_message = on_message,
    on_error = on_error,
    on_close = on_close
)
ws.on_open = on_open
ws.run_forever()

--- CONNECTED --- 
